In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly as py
import pandas as pd
from chart_studio.plotly import plot, iplot

# from plotly.offline import init_notebook_mode, iplot
from tqdm import tqdm_notebook

from scvi.dataset import PbmcDataset, GeneExpressionDataset
from scvi.models import VAE, IAVAE
from scvi.inference import UnsupervisedTrainer
from scvi.utils import demultiply, make_dir_if_necessary, predict_de_genes, save_fig, save_pickle, load_pickle
from scvi_utils import estimate_de_proba, estimate_lfc_density, estimate_lfc_mean
from R_interop import all_predictions, all_de_predictions


N_EPOCHS = 100
DELTA = 0.5
# SIZES = [5, 10, 20, 30, 50, 100]
SIZE = 100
SIZES = [SIZE]
DO_CLOUD = True
N_SIZES = len(SIZES)

Q0 = 5e-2
N_TRAININGS = 5
N_PICKS = 10

np.random.seed(42)

PATH_TO_SCRIPTS = "/home/ubuntu/conquer_comparison/scripts"

In [2]:
import chart_studio.plotly as py

py.sign_in("pierreboyeau", "2wvdnWZ2Qut1zD07ADVy")

# Import Dataset and Training scVI-based models

In [3]:
DIR_PATH = "lfc_estimates/null"
make_dir_if_necessary(DIR_PATH)

## PBMC Dataset

In [4]:
dataset_path = os.path.join(DIR_PATH, "dataset.pickle")
if not os.path.exists(dataset_path):
    all_dataset = PbmcDataset()
    all_dataset.subsample_genes(2000)
    save_pickle(all_dataset, filename=dataset_path)
else:
    all_dataset = load_pickle(filename=dataset_path)

unique_elements, counts_elements = np.unique(
    all_dataset.labels.squeeze(), return_counts=True
)

df = pd.DataFrame(dict(counts=counts_elements, cell_types=all_dataset.cell_types))
px.scatter(df, y="counts", x="cell_types")

mask = all_dataset.labels.squeeze() == 2
# By default all cells are labelled 2
fake_labels = 2.0 * np.ones(len(all_dataset))
# Except cluster 2 that is either 0 or 1
fake_labels[mask] = np.random.random(mask.sum()) >= 0.5

dataset = GeneExpressionDataset()
dataset.populate_from_data(
    X=all_dataset.X,
    labels=fake_labels,
    batch_indices=all_dataset.batch_indices,
)

n_genes = dataset.nb_genes
is_significant_de = np.zeros(n_genes, dtype=bool)

INFO:scvi.dataset.dataset:batch 0
INFO:scvi.dataset.dataset:Local library mean : [[7.1966677]]
INFO:scvi.dataset.dataset:Local library std: [[0.15904291]]
INFO:scvi.dataset.dataset:batch 1
INFO:scvi.dataset.dataset:Local library mean : [[7.1491838]]
INFO:scvi.dataset.dataset:Local library std: [[0.15288582]]
INFO:scvi.dataset.dataset:Remapping labels to [0,N]
INFO:scvi.dataset.dataset:Remapping batch_indices to [0,N]


In [31]:
all_dataset.cell_types

array(['B cells', 'CD14+ Monocytes', 'CD4 T cells', 'CD8 T cells',
       'Dendritic Cells', 'FCGR3A+ Monocytes', 'Megakaryocytes',
       'NK cells', 'Other'], dtype=object)

In [5]:
print(np.unique(dataset.labels.squeeze()))

[0 1 2]


## Save data

In [6]:
n_examples = len(dataset)
labels = dataset.labels.squeeze()
interesting_indices = np.where((labels == 0) | (labels == 1))[0]
TEST_INDICES = np.random.permutation(interesting_indices)[:1001]

x_test, y_test = dataset.X[TEST_INDICES, :], dataset.labels[TEST_INDICES, :].squeeze()
data_path = os.path.join(DIR_PATH, 'data.npy')
labels_path = os.path.join(DIR_PATH, 'labels.npy')

np.save(
    data_path,
    np.array(x_test.todense()).squeeze().astype(int)
)
np.savetxt(
    labels_path,
    y_test.squeeze()
)

## Train parameters

In [7]:
mdl_params = dict(
    iaf=dict(n_hidden=128, n_layers=1, do_h=True, n_latent=10, t=4),
    mf=dict(n_hidden=128, n_layers=1, n_latent=10),
    iaf_k5=dict(n_hidden=128, n_layers=1, do_h=True, n_latent=10, t=4),
    mf_k5=dict(n_hidden=128, n_layers=1, n_latent=10),
)
train_params = dict(
    iaf=dict(ratio_loss=True, test_indices=TEST_INDICES),
    mf=dict(ratio_loss=True, test_indices=TEST_INDICES),
    iaf_k5=dict(ratio_loss=True, test_indices=TEST_INDICES, k_importance_weighted=5),
    mf_k5=dict(ratio_loss=True, test_indices=TEST_INDICES, k_importance_weighted=5)
)
train_fn_params = dict(
    iaf=dict(n_epochs=N_EPOCHS, lr=1e-3),
    mf=dict(n_epochs=N_EPOCHS, lr=1e-3),
    iaf_k5=dict(n_epochs=N_EPOCHS, lr=1e-3),
    mf_k5=dict(n_epochs=N_EPOCHS, lr=1e-3),
)

# Competitors

In [8]:
other_predictions = all_predictions(
    filename=os.path.join(DIR_PATH, "all_predictions.pickle"),
    n_genes=n_genes, 
    n_picks=N_PICKS, 
    sizes=SIZES, 
    data_path=data_path, 
    labels_path=labels_path,
    path_to_scripts=PATH_TO_SCRIPTS
)

other_predictions = all_de_predictions(
    other_predictions, significance_level=Q0, delta=DELTA
)

In [9]:
# # Modif DESeq2
# from R_interop import NDESeq2
# from tqdm import tqdm

# lfcs_deseq2 = np.zeros((N_SIZES, N_PICKS, n_genes))
# pvals_deseq2 = np.zeros((N_SIZES, N_PICKS, n_genes))
# for (size_ix, size) in enumerate(tqdm(SIZES)):
#     for exp in range(N_PICKS):
#         deseq_inference = NDESeq2(
#             A=size,
#             B=size,
#             data=data_path,
#             labels=labels_path,
#             cluster=(0, 1),
#             path_to_scripts=PATH_TO_SCRIPTS,
#             lfc_threshold=DELTA
#         )
#         res_df = deseq_inference.fit()
#         lfcs_deseq2[size_ix, exp, :] = res_df["lfc"].values
#         pvals_deseq2[size_ix, exp, :] = res_df["padj"].values
# deseq_res = dict(lfc=lfcs_deseq2.squeeze(), pval=pvals_deseq2.squeeze())

# # from scvi.utils import save_pickle

# # print(deseq_res['pval'].shape)
# # print(other_predictions['edger']['pval'].shape)

# # other_predictions["deseq2"] = deseq_res
# # save_pickle(data=other_predictions, filename=os.path.join(DIR_PATH, "all_predictions.pickle"))

In [10]:
# from scvi.utils import save_pickle

# print(deseq_res['pval'].shape)
# print(other_predictions['edger']['pval'].shape)

# other_predictions["deseq2"] = deseq_res
# save_pickle(data=other_predictions, filename=os.path.join(DIR_PATH, "all_predictions.pickle"))

# Experiments

## LFC Error

In [11]:
lfcs_scVI = estimate_lfc_mean(
    filename=os.path.join(DIR_PATH, "lfc_means_mf.pickle"),
    mdl_class=VAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
    sizes=[SIZE],
    n_picks=N_PICKS,
)[SIZE]

lfcs_scVI_ia = estimate_lfc_mean(
    filename=os.path.join(DIR_PATH, "lfc_means_iaf.pickle"),
    mdl_class=IAVAE,
    dataset=dataset,
    mdl_params=mdl_params["iaf"],
    train_params=train_params["iaf"],
    train_fn_params=train_fn_params["iaf"],
    sizes=[SIZE],
    n_picks=N_PICKS,
)[SIZE]

In [12]:
print(other_predictions["mast"]["lfc"].shape)

lfcs_deseq2 = other_predictions["deseq2"]["lfc"]
lfcs_edger = other_predictions["edger"]["lfc"]
lfcs_mast = other_predictions["mast"]["lfc"]
assert lfcs_mast.shape == (N_PICKS, n_genes), lfcs_mast.shape

(10, 2000)


In [13]:
print(lfcs_deseq2.shape)
print(lfcs_edger.shape)
print(lfcs_mast.shape)
print(lfcs_scVI.shape)
print(lfcs_scVI_ia.shape)

(10, 2000)
(10, 2000)
(10, 2000)
(10, 2000)
(10, 2000)


In [39]:
lfcs_scVI[np.isinf(lfcs_scVI)] = 0.0
lfcs_scVI_ia[np.isinf(lfcs_scVI_ia)] = 0.0
lfcs_deseq2[np.isinf(lfcs_deseq2)] = 0.0
lfcs_edger[np.isinf(lfcs_edger)] = 0.0
lfcs_mast[np.isinf(lfcs_mast)] = 0.0

In [40]:
def l2_err(vals):
    res = 0.5 * (vals ** 2) ** (0.5)
    res = np.nanmean(res, axis=-1)
    return res


scVI_errs = l2_err(lfcs_scVI)
scVI_ia_errs = l2_err(lfcs_scVI_ia)
deseq2_errs = l2_err(lfcs_deseq2)
edger_errs = l2_err(lfcs_edger)
mast_errs = l2_err(lfcs_mast)

trace1 = go.Box(y=scVI_errs, name="scVI")
trace2 = go.Box(y=scVI_ia_errs, name="scVI IAF")
trace3 = go.Box(y=deseq2_errs, name="DeSeq2")
trace4 = go.Box(y=edger_errs, name="edgeR")
trace5 = go.Box(y=mast_errs, name="MAST")
traces = [trace1, trace2, trace3, trace4, trace5]

layout = go.Layout(title="L2 Error on Null Real Data")


fig = go.Figure(traces, layout=layout)
# save_fig(fig, filename="pbmc_null_lfc_err", do_cloud=DO_CLOUD)
iplot(fig, filename="pbmc_null_lfc_err")

In [43]:
round(12.202020202303, 3)

12.202

In [49]:
def formater(mean, std):
    return "${0:.3f} \pm {1:.3f}$".format(mean, std)

vals = [
    formater(scVI_errs.mean(), scVI_errs.std()),
    formater(scVI_ia_errs.mean(), scVI_ia_errs.std()),
    formater(deseq2_errs.mean(), deseq2_errs.std()),
    formater(edger_errs.mean(), edger_errs.std()),
    formater(mast_errs.mean(), mast_errs.std()),
]

index = [
    "scVI",
    "scVI",
    "DeSeq2",
    "edgeR",
    "MAST",
]

latex_df = pd.DataFrame({"L2 Error": vals}, index=index)
print(latex_df.to_latex(escape=False))

\begin{tabular}{ll}
\toprule
{} &           L2 Error \\
\midrule
scVI   &  $0.020 \pm 0.005$ \\
scVI   &  $0.019 \pm 0.010$ \\
DeSeq2 &  $0.078 \pm 0.007$ \\
edgeR  &  $0.112 \pm 0.003$ \\
MAST   &  $0.022 \pm 0.002$ \\
\bottomrule
\end{tabular}



In [35]:
from scipy.stats import mannwhitneyu, ttest_1samp
import plotly.figure_factory as ff


def a_better_b_sign(a, b):
    _, p = mannwhitneyu(a, b, alternative="less")
    return p


vals = [scVI_errs, scVI_ia_errs, deseq2_errs, edger_errs, mast_errs]

x = ["scVI", "scVI IAF", "DeSeq2", "edgeR", "MAST"]

mat = [[a_better_b_sign(b, a) for a in vals] for b in vals]

fig = ff.create_annotated_heatmap(z=mat, x=x, y=x)
fig = fig.update_layout(
    title_text="P values for error test on null data (pval that line a better than col b)"
)

# save_fig(fig, filename="pbmc_null_lfc_err", do_cloud=DO_CLOUD)
iplot(fig, filename="pbmc_null_lfc_err_sign")

## Significant DE genes

In [16]:
os.listdir(DIR_PATH)

['de_probas_mf.pickle.npy',
 'data.npy',
 'labels.npy',
 'all_predictions.pickle',
 'de_probas_iaf.pickle.npy',
 'lfc_means_iaf.pickle',
 'dataset.pickle',
 'lfc_means_mf.pickle']

In [17]:
de_probas_mf = estimate_de_proba(
    filename=os.path.join(DIR_PATH, "de_probas_mf.pickle.npy"),
    mdl_class=VAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
    sizes=[SIZE],
    n_trainings=N_TRAININGS,
    n_picks=N_PICKS,
).squeeze()

de_probas_ia = estimate_de_proba(
    filename=os.path.join(DIR_PATH, "de_probas_iaf.pickle.npy"),
    mdl_class=IAVAE,
    dataset=dataset,
    mdl_params=mdl_params["iaf"],
    train_params=train_params["iaf"],
    train_fn_params=train_fn_params["iaf"],
    sizes=[SIZE],
    n_trainings=N_TRAININGS,
    n_picks=N_PICKS,
).squeeze()

In [26]:
def compute_nb_predicted(probas_arr):
#     nb_predicted = np.zeros((N_TRAININGS, N_PICKS))
#     for i in range(N_TRAININGS):
#         for k in range(N_PICKS):
#             probs_pred_de = probas_arr[i, k, :]
#             is_pred_de = predict_de_genes(probs_pred_de, desired_fdr=Q0)
#             nb_predicted[i, j, k] = is_pred_de.sum()
    
    nb_predicted = np.zeros((N_PICKS))
    for k in range(N_PICKS):
        probs_pred_de = probas_arr[k, :]
        is_pred_de = predict_de_genes(probs_pred_de, desired_fdr=Q0)
        nb_predicted[k] = is_pred_de.sum()
    return n_genes - nb_predicted

def naive_is_de(probas_arr):
    probas_arr[np.isnan(probas_arr)] = 0.0
    return (probas_arr <= Q0).sum(-1)

In [21]:
de_probas_mf.shape

(5, 10, 2000)

In [22]:
other_predictions['deseq2']['pval'].shape

(10, 2000)

In [27]:
nb_predicted_scVI = compute_nb_predicted(de_probas_mf[0])
nb_predicted_scVI_ia = compute_nb_predicted(de_probas_ia[0])

nb_predicted_deseq2 = naive_is_de(other_predictions['deseq2']['pval'])
nb_predicted_edger = naive_is_de(other_predictions['edger']['pval'])
nb_predicted_mast = naive_is_de(other_predictions['mast']['pval'])

In [28]:
nb_predicted_scVI

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [25]:
nb_predicted_deseq2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
bounds_scVI = [nb_predicted_scVI.min(), nb_predicted_scVI.max()]
bounds_scVI_ia = [nb_predicted_scVI_ia.min(), nb_predicted_scVI_ia.max()]
bounds_deseq2 = [nb_predicted_deseq2.min(), nb_predicted_deseq2.max()]
bounds_edger = [nb_predicted_edger.min(), nb_predicted_edger.max()]
bounds_mast = [nb_predicted_mast.min(), nb_predicted_mast.max()]

In [29]:
pd.DataFrame(
    dict(
        mean_predicted=[
            nb_predicted_scVI.mean(), 
            nb_predicted_scVI_ia.mean(), 
            nb_predicted_deseq2.mean(), 
            nb_predicted_edger.mean(), 
            nb_predicted_mast.mean()
        ],
    ),
    index=['MF', 'IAF', 'DESeq2', 'EdgeR', 'MAST']
).T

,MF,IAF,DESeq2,EdgeR,MAST
mean_predicted,1.0,1.0,0.0,0.2,65.5


# Old

In [ ]:
# scVI_robustness = get_robustness(lfcs_scVI)
# scVI_ia_robustness = get_robustness(lfcs_scVI_ia)
# deseq2_robustness = get_robustness(lfcs_deseq2)
# edge_r_robustness = get_robustness(lfcs_edge_r)
# mast_robustness = get_robustness(lfcs_mast)

# trace1 = go.Box(y=scVI_robustness, name="scVI")
# trace2 = go.Box(y=scVI_ia_robustness, name="scVI IAF")
# trace3 = go.Box(y=deseq2_robustness, name="DeSeq2")
# trace4 = go.Box(y=edge_r_robustness, name="edgeR")
# trace5 = go.Box(y=mast_robustness, name="MAST")
# traces = [trace1, trace2, trace3, trace4, trace5]

# layout = go.Layout(title="Robustness on Null Real Data")

# fig = go.Figure(traces, layout=layout)
# # fig.show()
# iplot(fig, filename='robustness_null_ercc_dataset')

In [ ]:
# from scipy.stats import mannwhitneyu, ttest_1samp
# import plotly.figure_factory as ff


# def a_better_b_sign(a, b):
#     _, p = mannwhitneyu(a, b, alternative="less")
#     return p


# vals = [
#     scVI_robustness,
#     scVI_ia_robustness,
#     deseq2_robustness,
#     edge_r_robustness,
#     mast_robustness,
# ]

# x = ["scVI", "scVI IAF", "DeSeq2", "edgeR", "MAST"]

# mat = [[a_better_b_sign(b, a) for a in vals] for b in vals]

# fig = ff.create_annotated_heatmap(z=mat, x=x, y=x)
# fig.update_layout(
#     title_text="P values for robustness test on null data (pval that line a better than col b)"
# )

# iplot(fig, filename='significance_robustness_null_ercc_dataset')